##Variability Metrics

These variabilty metrics were developed as a first attempt to understand LSST performance for discovering and characterising periodic variables. They are available in the [sims_maf_contrib](https://github.com/LSST-nonproject/sims_maf_contrib) github repo, in [varMetrics.py](https://github.com/LSST-nonproject/sims_maf_contrib/blob/master/mafContrib/varMetrics.py). 

One metric (PhaseGaps) looks at the gaps in phase coverage for a set of observations. At each point in the sky (if we use a healpix slicer), a range of periods and the times of the visits are converted to phase values: then the largest gap in these phases is calculated. The metric calculates the max gap for a range of periods at each point; thus it also provides a few reduce functions to convert this to the period with the largest gap (at each point in the sky) and the value of the largest gap in phase coverage (at each point in the sky). We can also read the metric values back into python and generate plots of phase gaps vs. period, including the data for all points.

The second metric (PeriodDeviation) generates a light curve (representing the variability of the source as a simple sine curve but neglecting photometric errors) with user-settable range of periods, then attempts to recover the period using a simple lomb-scargle calculation. Changing the period fitting routine would doubtless improve the period recovery.  Again, this should be paired with a healpix slicer to evaluate sets of visits which correspond to the same spot on the sky, and again each point is sampled at a range of different periods so the metric provides reduce methods and we can also read back in the results to evaluate true period vs. fit period across the entire sky. 

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
# Import MAF modules.
import lsst.sims.maf.db as db
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.stackers as stackers
from lsst.sims.maf.metricBundles import MetricBundle, MetricBundleGroup
# Import the contributed metrics and stackers 
import mafContrib

Connect to the opsim database and set up some metadata.

In [2]:
runName = 'enigma_1189'
dbAddress = 'sqlite:///' + runName + '_sqlite.db'
opsdb = db.OpsimDatabase(dbAddress)
outDir = 'variability_test'

Set up the metrics and slicer. 

In [3]:
phaseMetric = mafContrib.PhaseGapMetric(nPeriods=5, periodMin=0.5, periodMax=3.5)
periodMetric = mafContrib.PeriodDeviationMetric(nPeriods=5, periodMin=0.5, periodMax=3.5)
slicer = slicers.HealpixSlicer(nside=64, lonCol='ditheredRA', latCol='ditheredDec')

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes


In [4]:
summaryMetrics = [metrics.MinMetric(), metrics.MaxMetric(), metrics.MeanMetric()]

In [5]:
sqlconstraint = 'night<365 and (filter="r" or filter="i")'
phaseBundle = MetricBundle(phaseMetric, slicer, sqlconstraint=sqlconstraint, 
                           runName=runName, summaryMetrics=summaryMetrics)
periodBundle = MetricBundle(periodMetric, slicer, sqlconstraint=sqlconstraint, 
                            runName=runName, summaryMetrics=summaryMetrics)

In [6]:
bdict = {'Phase':phaseBundle, 'Period':periodBundle}

In [7]:
resultsDb = db.ResultsDb(outDir=outDir)
bgroup = MetricBundleGroup(bdict, opsdb, outDir=outDir, resultsDb=resultsDb)

In [ ]:
bgroup.runAll()

Querying database with constraint night<365 and (filter="r" or filter="i")
Found 84922 visits